In [2]:
# installing modules 
!pip3 install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 11.0 MB/s eta 0:00:00


In [52]:
import pandas as pd 
import numpy as np


from sklearn. preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

#PyTorch
import torch
import torch.nn as nn 
from tqdm import tqdm

import torchvision as tv 
import torch.nn.functional as F

In [26]:
#data downloading 

train = pd.read_csv('/content/train.csv')
X_test = pd.read_csv('/content/test.csv')


# Fill missing values (Nan) with 0 
train.fillna(0, inplace = True)
X_test.fillna(0 , inplace= True)


In [5]:

# separate y_train / X_train from train 
y_train = train['Transported']
X_train = train.drop(columns = ['Transported'],axis = 1)


X_train


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre


In [6]:
#Let's separate the Name column into a variable and encode the names into unique numbers

name = X_train['Name']
X_train = X_train.drop( columns = ['Name'] , axis = 1 )
labelencoder = LabelEncoder()
name = name.astype('str')
name = labelencoder.fit_transform(name)
name_df = pd.DataFrame(name)

In [7]:
# Split the column "PassengerId" into 2 new columns in ds : Passenger and Number_in_group

passengers = []
ids = []

Pass_id_dict = {}
for i in X_train.PassengerId :
  split_i = i.split('_')
  split_i[1] = int(split_i[1])
  
  passengers.append(split_i[0])
  ids.append(split_i[1])
  
X_train['Group'] = passengers
X_train['Number_in_group'] = ids


In [8]:
#Delete old column from ds 
X_train = X_train.drop( columns = ['PassengerId'] , axis = 1 )


In [9]:
#split the column cabin into 3 new columns : Deck / Num / Side 

deck =[]
num=[]
side=[]

for i in X_train.Cabin :
  if i != 0 :
    
    split_i = i.split('/')
    deck.append(split_i[0])
    num.append(int(split_i[1]))
    side.append(split_i[2])
  if i == 0 :
    deck.append(0)
    num.append(0)
    side.append(0)



  
X_train['Deck'] = deck
X_train['Num'] = num
X_train['Side'] = side

# Delete old one 
X_train = X_train.drop( columns = ['Cabin' ] , axis = 1 )





In [10]:
# Group categorical columns to use OneHotEncoding

categorical = X_train[['HomePlanet' , 'Destination'  , 'Deck' , 'Side' ]]
categorical = pd.get_dummies(categorical)


In [11]:
#Detele columns after One Hot Encoding where unique value is Nan ( 0 after fillna )

categorical = categorical.drop( columns = ['HomePlanet_0' , 'Destination_0' , 'Deck_0' , 'Side_0'] , axis = 1)


In [14]:
# Transform numerical columns to type int 

numerical = X_train[['CryoSleep' , 'Age' , 'VIP' , 'RoomService' , 'FoodCourt' , 'ShoppingMall' , 'Spa' , 'VRDeck' , 'Group' , 'Number_in_group' , 'Num']]
numerical = numerical.astype('int')


# Join cat columns and num columns
X_train = categorical.join(numerical)

In [15]:
# join the columns "Name" too

X_train = X_train.join(name_df)
X_train = X_train.rename( columns = { 0 : 'Name'  })

In [16]:
X_train

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,...,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Number_in_group,Num,Name
0,0,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,1,0,5253
1,1,0,0,0,0,1,0,0,0,0,...,0,109,9,25,549,44,2,1,0,4503
2,0,1,0,0,0,1,1,0,0,0,...,1,43,3576,0,6715,49,3,1,0,458
3,0,1,0,0,0,1,1,0,0,0,...,0,0,1283,371,3329,193,3,2,0,7150
4,1,0,0,0,0,1,0,0,0,0,...,0,303,70,151,565,2,4,1,1,8320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,1,0,1,0,0,1,0,0,0,...,1,0,6819,0,1643,74,9276,1,98,3525
8689,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,9278,1,1499,4781
8690,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1872,1,0,9279,1,1500,3003
8691,0,1,0,1,0,0,0,0,0,0,...,0,0,1049,0,353,3235,9280,1,608,1597


In [17]:
# all our DataFrame consists now of digits only

In [18]:
y_train

0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 8693, dtype: bool

In [19]:
# Transform y_train from bool to int

y_train = y_train.astype('int')



In [20]:
# Using of Standard Scaler to convert all values to a normal distribution with mean 0 and standart deviation 1

scaler = StandardScaler()
names = X_train.columns 
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame( X_train , columns=names)
 

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,...,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Number_in_group,Num,Name
0,-1.060617,1.754795,-0.503664,-0.511013,-0.317487,0.685313,-0.174191,3.187347,-0.30661,-0.241218,...,-0.153063,-0.333105,-0.281027,-0.283579,-0.270626,-0.263003,-1.734409,-0.491161,-1.141624,0.446353
1,0.942847,-0.569867,-0.503664,-0.511013,-0.317487,0.685313,-0.174191,-0.313741,-0.30661,-0.241218,...,-0.153063,-0.168073,-0.275387,-0.241771,0.217158,-0.224205,-1.734034,-0.491161,-1.141624,0.146306
2,-1.060617,1.754795,-0.503664,-0.511013,-0.317487,0.685313,5.740821,-0.313741,-0.30661,-0.241218,...,6.533255,-0.268001,1.959998,-0.283579,5.695623,-0.219796,-1.733660,-0.491161,-1.141624,-1.471948
3,-1.060617,1.754795,-0.503664,-0.511013,-0.317487,0.685313,5.740821,-0.313741,-0.30661,-0.241218,...,-0.153063,-0.333105,0.523010,0.336851,2.687176,-0.092818,-1.733660,0.457443,-1.141624,1.205272
4,0.942847,-0.569867,-0.503664,-0.511013,-0.317487,0.685313,-0.174191,-0.313741,-0.30661,-0.241218,...,-0.153063,0.125652,-0.237159,-0.031059,0.231374,-0.261240,-1.733286,-0.491161,-1.139678,1.673345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,-1.060617,1.754795,-0.503664,1.956897,-0.317487,-1.459188,5.740821,-0.313741,-0.30661,-0.241218,...,6.533255,-0.333105,3.992336,-0.283579,1.189173,-0.197751,1.738236,-0.491161,-0.950907,-0.244955
8689,0.942847,-0.569867,-0.503664,-0.511013,3.149739,-1.459188,-0.174191,-0.313741,-0.30661,-0.241218,...,-0.153063,-0.333105,-0.281027,-0.283579,-0.270626,-0.263003,1.738984,-0.491161,1.775567,0.257523
8690,0.942847,-0.569867,-0.503664,-0.511013,-0.317487,0.685313,-0.174191,-0.313741,-0.30661,-0.241218,...,-0.153063,-0.333105,-0.281027,2.846999,-0.269737,-0.263003,1.739359,-0.491161,1.777513,-0.453788
8691,-1.060617,1.754795,-0.503664,1.956897,-0.317487,-1.459188,-0.174191,-0.313741,-0.30661,-0.241218,...,-0.153063,-0.333105,0.376365,-0.283579,0.043013,2.589576,1.739733,-0.491161,0.041600,-1.016276


In [45]:


#Prefit train data to load it into dataloader ( from df to (row , target ))
X_train = X_train.join(y_train)
train_list = []

for i, row in X_train.iterrows():
    row = row.tolist()
    label = row[-1]
    label = int(label)
    
    info = row[:-1]

    info = torch.Tensor(info)
    
    tpl = ( info , label )
    
    train_list.append(tpl)
    




In [46]:
#DataLoader 

batch_size = 16
dataloader = torch.utils.data.DataLoader( train_list  ,
                                         batch_size = batch_size ,
                                         shuffle = True , 
                                         num_workers= 0 , 
                                         drop_last = True 
                                         )

In [47]:
#Architecture of my NN 

class Neural_network(nn.Module):
  
  def __init__(self):
       
        super().__init__() 
        
        self.flat = nn.Flatten()
        self.linear1 = nn.Linear(28 , 512)
        self.linear2 = nn.Linear(512 , 256)
        self.linear3 = nn.Linear(256 , 128)
        self.linear4 = nn.Linear(128, 64)
        self.linear5 = nn.Linear(64 , 128)
        self.linear6 = nn.Linear( 128 , 128)
        self.linear7 = nn.Linear( 128 , 2 )

        self.act = nn.ReLU()
        
       #
  def forward(self, x): # x - tensor 
    out = self.flat(x)
    out = self.linear1(out)
    out = self.act(out)
    out = self.linear2(out)
    out = self.act(out)
    out = self.linear3(out)
    out = self.act(out)
    out = self.linear4(out)
    out = self.act(out)
    out = self.linear5(out)
    out = self.act(out)
    out = self.linear6(out)
    out = self.act(out)
    out = self.linear7(out)

    return out

In [48]:
# Model , loss , optimizer

model = Neural_network()
loss_fn =  torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW( model.parameters() , lr = 4e-3 )

In [50]:
# function of calculating accuracy on train

def accuracy(pred , label) :
    
    answer = F.softmax(pred.detach()).numpy().argmax(1)  == label.numpy().argmax(1)
    
    return answer.mean()

In [53]:
# Model training

epochs = 20 

for epoch in range(epochs):

  loss_val = 0 
  acc_val = 0 

  for sample , label in (pbar := tqdm(dataloader)) : # tqdm for vizualization of progress
    optimizer.zero_grad() #  refresh gradients
     
    label = nn.functional.one_hot( label , 2 ).float() # 1 -> [1,0]
    
    pred = model(sample) # vector of probabilities 
    
    loss = loss_fn(pred , label) 


    loss.backward() # calculate gradients 

    loss_item = loss.item()
    loss_val += loss_item

    acc_current = accuracy(pred , label)
    acc_val  += acc_current
    optimizer.step() 

  

    pbar.set_description(f'loss : {loss_item:.4f}\taccuracy : {acc_current:.3f}')  
  print( f'epoch {epoch + 1 }' , f' loss_val : {loss_val / len(dataloader) }'
  , f'acc_val : {acc_val / len(dataloader)}')

  0%|          | 0/543 [00:00<?, ?it/s]<ipython-input-50-b730810a7890>:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  answer = F.softmax(pred.detach()).numpy().argmax(1)  == label.numpy().argmax(1)
loss : 0.4312	accuracy : 0.750: 100%|██████████| 543/543 [00:04<00:00, 126.78it/s]


epoch 1  loss_val : 0.46777766610926247 acc_val : 0.765768876611418


loss : 0.4791	accuracy : 0.750: 100%|██████████| 543/543 [00:04<00:00, 125.42it/s]


epoch 2  loss_val : 0.43772269535679403 acc_val : 0.7891344383057091


loss : 0.4247	accuracy : 0.812: 100%|██████████| 543/543 [00:04<00:00, 134.84it/s]


...

epoch 18  loss_val : 0.35216540252553163 acc_val : 0.8261970534069981


loss : 0.3124	accuracy : 0.875: 100%|██████████| 543/543 [00:04<00:00, 130.32it/s]


epoch 19  loss_val : 0.3451726057774235 acc_val : 0.8278084714548803


loss : 0.3487	accuracy : 0.875: 100%|██████████| 543/543 [00:04<00:00, 132.30it/s]


epoch 20  loss_val : 0.34977991830962357 acc_val : 0.8244705340699816


In [27]:
# Use the code we used to prepare the train data for the test data 

passenger_id = X_test['PassengerId'] 

In [28]:
# test
name = X_test['Name']
X_test = X_test.drop(columns =['Name'] , axis = 1 )

labelencoder = LabelEncoder()
name = name.astype('str')
name = labelencoder.fit_transform(name)
name_df = pd.DataFrame(name)

passengers = []
ids = []
Pass_id_dict = {}
for i in X_test.PassengerId :
  split_i = i.split('_')
  split_i[1] = int(split_i[1])
  
  passengers.append(split_i[0])
  ids.append(split_i[1]) 
X_test['Group'] = passengers
X_test['Number_in_group'] = ids

X_test = X_test.drop( columns = ['PassengerId'] , axis = 1 )



import numpy as np
deck =[]
num=[]
side=[]
for i in X_test.Cabin :
  if i != 0 :
    
    split_i = i.split('/')
    deck.append(split_i[0])
    num.append(int(split_i[1]))
    side.append(split_i[2])
  if i == 0 :
    deck.append(0)
    num.append(0)
    side.append(0)



  
X_test['Deck'] = deck
X_test['Num'] = num
X_test['Side'] = side
X_test = X_test.drop( columns = ['Cabin' ] , axis = 1 )

categorical = X_test[['HomePlanet' , 'Destination'  , 'Deck' , 'Side']]
categorical = pd.get_dummies(categorical)




numerical = X_test[['CryoSleep' , 'Age' , 'VIP' , 'RoomService' , 'FoodCourt' , 'ShoppingMall' , 'Spa' , 'VRDeck' , 'Group' , 'Number_in_group' , 'Num']]
categorical = categorical.drop( columns = ['HomePlanet_0' , 'Destination_0' , 'Deck_0' , 'Side_0'] , axis = 1)
numerical = numerical.astype('int')

X_test = categorical.join(numerical)

X_test = X_test.join(name_df)
X_test = X_test.rename( columns = { 0 : 'Name'  })
X_test

,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,...,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Number_in_group,Num,Name
0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,13,1,3,2913
1,1,0,0,0,0,1,0,0,0,0,...,0,0,9,0,2823,0,18,1,4,2407
2,0,1,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,19,1,0,3377
3,0,1,0,0,0,1,0,0,1,0,...,0,0,6652,0,181,585,21,1,1,2712
4,1,0,0,0,0,1,0,0,0,0,...,0,10,0,635,0,0,23,1,5,669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,9266,2,1496,2106
4273,1,0,0,0,0,1,0,0,0,0,...,0,0,847,17,10,144,9269,1,0,2655
4274,0,0,1,1,0,0,0,0,0,1,...,0,0,0,0,0,0,9271,1,296,2086
4275,0,1,0,0,0,0,0,0,0,1,...,0,0,2680,0,0,523,9273,1,297,2328


In [29]:
# Standart Scaler for test data

scaler = StandardScaler()
names = X_test_f.columns 
X_test = scaler.fit_transform(X_test_f)
X_test = pd.DataFrame( X_test , columns=names)



In [30]:
X_test


,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Deck_A,Deck_B,Deck_C,Deck_D,...,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Group,Number_in_group,Num,Name
0,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.703425,-0.489847,-1.146527,0.708477
1,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.277879,-0.312173,2.287504,-0.246712,-1.701584,-0.489847,-1.144593,0.297314
2,-1.060016,1.807889,-0.525314,2.021290,-0.315862,-1.495894,-0.153136,-0.30408,3.323836,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,-1.701215,-0.489847,-1.152328,1.085511
3,-1.060016,1.807889,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,3.323836,-0.244898,...,-0.132689,-0.357339,4.121518,-0.312173,-0.104002,0.226648,-1.700479,-0.489847,-1.150395,0.545149
4,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.340723,-0.283840,0.832122,-0.267841,-0.246712,-1.699743,-0.489847,-1.142660,-1.114938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,1.703574,0.492373,1.740519,0.052730
4273,0.943382,-0.553131,-0.525314,-0.494734,-0.315862,0.668497,-0.153136,-0.30408,-0.300857,-0.244898,...,-0.132689,-0.357339,0.277095,-0.281538,-0.258790,-0.130193,1.704679,-0.489847,-1.152328,0.498833
4274,-1.060016,-0.553131,1.903624,2.021290,-0.315862,-1.495894,-0.153136,-0.30408,-0.300857,4.083326,...,-0.132689,-0.357339,-0.283840,-0.312173,-0.267841,-0.246712,1.705415,-0.489847,-0.579947,0.036478
4275,-1.060016,1.807889,-0.525314,-0.494734,-0.315862,-1.495894,-0.153136,-0.30408,-0.300857,4.083326,...,-0.132689,-0.357339,1.491019,-0.312173,-0.267841,0.176479,1.706152,-0.489847,-0.578013,0.233121


In [54]:
# prefit test Df to test dataloader
test_ds = []
for i, row in X_test.iterrows():
    row = row.tolist()
    row = torch.Tensor(row)
    test_ds.append(row)

In [56]:
batch_size = 16
test_dataloader = torch.utils.data.DataLoader( test_ds  ,
                                         batch_size = batch_size ,
                                         shuffle = False  , 
                                         num_workers= 0 , 
                                         
                                         )
predictions_list = []
for i in test_dataloader :
  pred = model(i)
  
  for j in pred :
    j = j.detach()
    j = F.softmax(j)
    j = j.numpy()
    j = j.argmax(0)
    predictions_list.append(j)

#make predictions

<ipython-input-56-aee69caaf7aa>:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  j = F.softmax(j)


In [57]:
result = pd.DataFrame({'PassengerId' : passenger_id , 'Transported' : predictions_list })  #convert it into new DF
result['Transported'] = result['Transported'].astype(bool)

In [58]:
result

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [59]:
result.to_csv ('/content/NN_predictions.csv', index= False ) 